In [1]:
from z3 import *
import cirq

In [2]:
#x = Int('x')
#y = Int('y')
#solve(x > 2, y < 10, x+2*y == 7)

X = [[Int("x_%s_%s" % (i,j)) for j in range(9)] for i in range(9)]
s = Solver()
s.add([And(X[i][j] >= 1, X[i][j] <= 9) for i in range(9) for j in range(9)])

s.add([Distinct([X[i][j] for i in range(9)]) for j in range(9)])
s.add([Distinct([X[i][j] for j in range(9)]) for i in range(9)])

s.add([Distinct([X[i+3*x][j+3*y] for i in range(3) for j in range(3)]) for x in range(3) for y in range(3)])

initial = (
    (0,0,0,0,9,4,0,3,0),
    (0,0,0,5,1,0,0,0,7),
    (0,8,9,0,0,0,0,4,0),
    (0,0,0,0,0,0,2,0,8),
    (0,6,0,2,0,1,0,5,0),
    (1,0,2,0,0,0,0,0,0),
    (0,7,0,0,0,0,5,2,0),
    (9,0,0,0,6,5,0,0,0),
    (0,4,0,9,7,0,0,0,0)
)

s.add([If(initial[i][j] == 0, True, X[i][j] == initial[i][j]) for i in range(9) for j in range(9)])

s.check()
m = s.model()

r = [[m.evaluate(X[i][j]) for j in range(9)] for i in range(9)]
print_matrix(r)

[[7, 1, 5, 8, 9, 4, 6, 3, 2],
 [2, 3, 4, 5, 1, 6, 8, 9, 7],
 [6, 8, 9, 7, 2, 3, 1, 4, 5],
 [4, 9, 3, 6, 5, 7, 2, 1, 8],
 [8, 6, 7, 2, 3, 1, 9, 5, 4],
 [1, 5, 2, 4, 8, 9, 7, 6, 3],
 [3, 7, 6, 1, 4, 8, 5, 2, 9],
 [9, 2, 8, 3, 6, 5, 4, 7, 1],
 [5, 4, 1, 9, 7, 2, 3, 8, 6]]


In [3]:
def test_circuit_a():
    n = 5
    qubits = cirq.LineQubit.range(n)
    circuit = cirq.Circuit()

    circuit.append([cirq.H(q) for q in qubits])

    entangled_pairs = [[0, 1], [1, 2], [1, 3], [3, 4]]
    circuit.append(
        [cirq.CZ(qubits[pair[0]], qubits[pair[1]]) for pair in entangled_pairs]
    )

    return circuit

test_circuit_a()

0: ───H───@───────────────
          │
1: ───H───@───@───@───────
              │   │
2: ───H───────@───┼───────
                  │
3: ───H───────────@───@───
                      │
4: ───H───────────────@───

In [ ]:
nof_qubits = 5
nof_stages = 23
nof_two_qubit_gates = 4
gate_targets = [(0,1), (0,2), (0,3), (0,4)]

# at stage s, qubit q is located at the interaction site (x_q_s, y_q_s)
# note: x_interaction_site[q][s] == x_q_s
x_interaction_site = [[Int("x_%s_%s" % (q,s)) for s in range(nof_stages)] for q in range(nof_qubits)]
y_interaction_site = [[Int("y_%s_%s" % (q,s)) for s in range(nof_stages)] for q in range(nof_qubits)]

# a_q_s == True iff qubit q is in an AOD trap at stage s
# note: array_index[q][s] == a_q_s
array_index = [[Bool("a_%s_%s" % (q,s)) for s in range(nof_stages)] for q in range(nof_qubits)]

# at stage s, qubit q is at AOD column c_q_s and at AOD row r_q_s 
# note: aod_col_indices[q][s] == c_q_s
aod_col_indices = [[Int("c_%s_%s" % (q,s)) for s in range(nof_stages)] for q in range(nof_qubits)]
aod_row_indices = [[Int("r_%s_%s" % (q,s)) for s in range(nof_stages)] for q in range(nof_qubits)]

# gate g_i is scheduled for stage t_i, gates are given chronologically
# note: time_coordinate[i] == t_i
time_coordinate = [Int("t_%s" % (g)) for g in range(nof_two_qubit_gates)]

constraints = []

# interaction sites are bounded by the device's max dimensions
max_dim_x = 8
max_dim_y = 8
constraints.extend([And(x_interaction_site[q][s] >= 0, x_interaction_site[q][s] < max_dim_x)
                    for q in range(nof_qubits) for s in range(nof_stages)])
constraints.extend([And(y_interaction_site[q][s] >= 0, y_interaction_site[q][s] < max_dim_y)
                    for q in range(nof_qubits) for s in range(nof_stages)])

# use only available AOD rows and columns
nof_aod_rows = 4
nof_aod_cols = 4
constraints.extend([And(aod_row_indices[q][s] >= 0, aod_row_indices[q][s] < nof_aod_rows)
                    for q in range(nof_qubits) for s in range(nof_stages)])
constraints.extend([And(aod_col_indices[q][s] >= 0, aod_col_indices[q][s] < nof_aod_cols)
                    for q in range(nof_qubits) for s in range(nof_stages)])

# if qubit q is in a SLM trap at stage s, it will be at the same position at stage s+1
constraints.extend([Implies(Not(array_index[q][s]),
                            And(x_interaction_site[q][s] == x_interaction_site[q][s+1],
                                y_interaction_site[q][s] == y_interaction_site[q][s+1]))
                    for q in range(nof_qubits) for s in range(nof_stages - 1)])

# gates are scheduled for stages in within stage timeframe
constraints.extend([And(time_coordinate[g] > 0,
                        time_coordinate[g] < nof_stages)
                    for g in range(nof_two_qubit_gates)])

# if we have a two-qubit gate g_i on qubits q1 and q2 (scheduled for t_i), then q1 and q2 must be on the same interaction site at stage t_i
constraints.extend([Implies(time_coordinate[g] == s,
                            And(x_interaction_site[gate_targets[g][0]][s] == x_interaction_site[gate_targets[g][1]][s],
                                y_interaction_site[gate_targets[g][0]][s] == y_interaction_site[gate_targets[g][1]][s]))
                    for g in range(nof_two_qubit_gates) for s in range(nof_stages)])

# qubits in AOD rows/cols at stage s will remain at the same rows/cols at stage s+1
constraints.extend([Implies(array_index[q][s],
                            And(aod_col_indices[q][s] == aod_col_indices[q][s+1],
                                aod_row_indices[q][s] == aod_row_indices[q][s+1]))
                    for q in range(nof_qubits) for s in range(nof_stages - 1)])

# AOD rows cannot cross each other when moved (a row below this one cannot have a qubit with higher y-value in its interaction_site position than a qubit on this row)
constraints.extend([Implies(And(aod_row_indices[q1][s] < aod_row_indices[q2][s],
                                And(q1 != q2,
                                And(array_index[q1][s], array_index[q2][s]))),
                            y_interaction_site[q1][s] <= y_interaction_site[q2][s])
                    for q1 in range(nof_qubits) for q2 in range(nof_qubits) for s in range(nof_stages)])
# AOD cols cannot cross each other when moved (a columm left of this one cannot have a qubit with higher x-value in its interaction_site position than a qubit on this column)
constraints.extend([Implies(And(aod_col_indices[q1][s] < aod_col_indices[q2][s],
                                And(q1 != q2,
                                And(array_index[q1][s], array_index[q2][s]))),
                            x_interaction_site[q1][s] <= x_interaction_site[q2][s])
                    for q1 in range(nof_qubits) for q2 in range(nof_qubits) for s in range(nof_stages)])


# interaction site ordering implies the same row/column ordering
constraints.extend([Implies(And(x_interaction_site[q1][s] < x_interaction_site[q2][s],
                                q1 != q2),
                            aod_col_indices[q1][s] < aod_col_indices[q2][s])
                    for q1 in range(nof_qubits) for q2 in range(nof_qubits) for s in range(nof_stages)])
constraints.extend([Implies(And(y_interaction_site[q1][s] < y_interaction_site[q2][s],
                                q1 != q2),
                            aod_row_indices[q1][s] < aod_row_indices[q2][s])
                    for q1 in range(nof_qubits) for q2 in range(nof_qubits) for s in range(nof_stages)])

# one qubit per trap
constraints.extend([Implies(And(array_index[q1][s], And(array_index[q2][s], q1 != q2)),
                            Or(aod_col_indices[q1][s] != aod_col_indices[q2][s],
                               aod_row_indices[q1][s] != aod_row_indices[q2][s]))
                    for q1 in range(nof_qubits) for q2 in range(nof_qubits) for s in range(nof_stages)])
constraints.extend([Implies(And(Not(array_index[q1][s]), And(Not(array_index[q2][s]), q1 != q2)),
                            Or(x_interaction_site[q1][s] != x_interaction_site[q2][s],
                               y_interaction_site[q1][s] != y_interaction_site[q2][s]))
                    for q1 in range(nof_qubits) for q2 in range(nof_qubits) for s in range(nof_stages)])


# gates are executed in chronological order given in the gates
constraints.extend([time_coordinate[g] < time_coordinate[g+1] for g in range(nof_two_qubit_gates-1)])

# at the final stage, each qubit is in its own interaction site
constraints.extend([Implies(q1 != q2,
                        Or(x_interaction_site[q1][nof_stages-1] != x_interaction_site[q2][nof_stages-1],
                            y_interaction_site[q1][nof_stages-1] != y_interaction_site[q2][nof_stages-1]))
                    for q1 in range(nof_qubits) for q2 in range(nof_qubits)])

# on each stage, if a qubit is not in a SLM trap, there must be a SLM trap on that interaction site
# NOT DONE: assume we have a SLM trap on each interaction site instead
#constraints.extend([Implies(Not(array_index[q][s]),
#                            )])


s = Solver()
for c in constraints:
    s.add(c)


In [41]:
print("nof stages:", nof_stages)
print(s.check())
m = "no model"
if str(s.check()) == 'sat':
    m = s.model()
    #print(m)

nof stages: 23
sat


In [48]:
stage = 0
for i in range(len(m)):
    as_str = str(m[i])
    splitted = as_str.split('_')
    stage_str = splitted[len(splitted)-1]
    if stage_str == str(stage):
        dec_ref = m[i]
        print(m[i], "->", m[dec_ref])

x_4_0 -> 6
a_0_0 -> False
x_2_0 -> 7
a_3_0 -> True
c_3_0 -> 2
a_2_0 -> False
c_4_0 -> 0
x_1_0 -> 7
c_2_0 -> 2
y_1_0 -> 3
x_0_0 -> 6
a_1_0 -> True
r_3_0 -> 0
c_1_0 -> 1
y_0_0 -> 2
t_0 -> 1
c_0_0 -> 0
r_1_0 -> 1
y_2_0 -> 2
y_4_0 -> 4
a_4_0 -> True
x_3_0 -> 7
r_0_0 -> 0
r_2_0 -> 0
y_3_0 -> 2
r_4_0 -> 3
